In [183]:
import requests
from bs4 import BeautifulSoup
import re
import uuid
import csv
import pandas as pd

In [184]:
df = pd.read_csv('fighter_links_UUID.csv')
df.head()

,link,UUID
0,http://www.ufcstats.com/fighter-details/c32aeb...,7e60e735-5d1d-4fa3-b7f7-bcc4a1f50c04
1,http://www.ufcstats.com/fighter-details/23024f...,cec78d13-b105-4e36-a908-2bcd35d211c9
2,http://www.ufcstats.com/fighter-details/480779...,84d443f9-78b9-4e4e-9b86-bd38270ccad1
3,http://www.ufcstats.com/fighter-details/94426b...,9f4d943c-191c-4a56-a20c-4286297f581b
4,http://www.ufcstats.com/fighter-details/49d2a0...,abbdfe77-12e0-40e4-b3dd-30bc906a5d71


In [185]:
df2 = pd.read_csv('event_UUID.csv')
df2.head()

,UUID,name,date,location,link
0,09f61ec0-1b0a-4c7d-9c99-90e20a0ae45d,UFC Fight Night: Burns vs. Morales,"May 17, 2025","Las Vegas, Nevada, USA",http://www.ufcstats.com/event-details/8ad022dd...
1,66a2d9bc-de67-43bf-87ee-b3db1b689797,UFC 315: Muhammad vs. Della Maddalena,"May 10, 2025","Montreal, Quebec, Canada",http://www.ufcstats.com/event-details/118463dd...
2,700f1515-41ab-4bdc-9a4f-7484296e3eaf,UFC Fight Night: Sandhagen vs. Figueiredo,"May 03, 2025","Des Moines, Iowa, USA",http://www.ufcstats.com/event-details/de20ffb3...
3,b931be39-e4dc-43b4-9942-88b108bb6007,UFC Fight Night: Machado Garry vs. Prates,"April 26, 2025","Kansas City, Missouri, USA",http://www.ufcstats.com/event-details/b2e3aca4...
4,52898274-33fa-49f0-adbf-77d912de1518,UFC 314: Volkanovski vs. Lopes,"April 12, 2025","Miami, Florida, USA",http://www.ufcstats.com/event-details/22f4b6cb...


In [186]:
events_UUID = df2['name'].values
opponent_keys = df.set_index(df.columns[0])[df.columns[1]].to_dict()
links = df['link'].values

In [187]:
url = 'http://www.ufcstats.com/fighter-details/476fe566d2df676e'
fighter_id = opponent_keys[url]
site = requests.get(url)
soup = BeautifulSoup(site.content, 'html.parser')
print(site)
print(fighter_id)

<Response [200]>
02e262c6-f245-4112-bc01-b2a6954f9bd9


In [188]:
height = None
reach = None
stance = None
dOB = None

details = soup.find('ul', class_='b-list__box-list')
lis = details.find_all('li')
for li in lis:
    desc = li.text.strip().split(':', 1)

    if desc[0].lower() == 'height':
        height = desc[1].lstrip()
    elif desc[0].lower() == 'reach':
        reach = desc[1].lstrip()
    elif desc[0].lower() == 'stance':
        stance = desc[1].lstrip()
    elif desc[0].lower() == 'dob':
        dOB = desc[1].lstrip()

print(height)
print(reach)
print(stance)
print(dOB)


6' 3"
78"
Switch
Feb 10, 1994


In [189]:
fights = soup.find_all('tr', class_=re.compile("b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click"))
fight_links = []
for fight in fights:
    fight_links.append(fight.find('a')['href'])
print(len(fight_links))

10


In [190]:
events = soup.find_all('td', class_=re.compile('b-fight-details__table-col l-page_align_left'))
print(len(events))

30


In [194]:
for i in range(len(fight_links)):
    #initalized before fight link
    opponent_id = None
    event_name = None
    event_date = None
    method_of_victory = None
    details = events[i*3:i*3+3]
    opponent_link = details[0].find_all('a')[1]['href']
    opponent_id = opponent_keys[opponent_link]

    event_info = details[1].find_all('p', class_='b-fight-details__table-text')
    event_name = event_info[0].text.strip()
    event_date = event_info[1].text.strip().replace('.', '')

    method_info = details[2].find_all('p', class_='b-fight-details__table-text')
    method_of_victory = f'{method_info[0].text.strip()}  {method_info[1].text.strip()}'

    if event_name not in events_UUID:
        continue


    name = None
    alias = None
    winner_id = None
    corner = None
    
    url = fight_links[i]
    site = requests.get(url)
    soup = BeautifulSoup(site.content, 'html.parser')
    
    details = soup.find_all('div', class_='b-fight-details__person')
    red_corner_info = details[0].find('a', class_='b-link b-fight-details__person-link')
    blue_corner_info = details[1].find('a', class_='b-link b-fight-details__person-link')

    #fighter is in red corner
    if opponent_keys[red_corner_info['href']] == fighter_id:
        corner = 'Red'
        name = red_corner_info.text.strip()
        alias = details[0].find('p', class_='b-fight-details__person-title').text.strip().replace('"', '')
        if details[0].find('i').text.strip() == "W":
            winner_id = fighter_id
        elif details[1].find('i').text.strip() =="W":
            winner_id = opponent_id
    #fighter is in blue corner
    else:
        corner = 'Blue'
        name = blue_corner_info.text.strip()
        alias = details[1].find('p', class_='b-fight-details__person-title').text.strip().replace('"', '')
        if details[1].find('i').text.strip() == "W":
            winner_id = fighter_id
        elif details[0].find('i').text.strip() =="W":
            winner_id = opponent_id


    details = soup.find('div', class_='b-fight-details__fight')
    weight = details.find('i', class_='b-fight-details__fight-title').text.strip().replace(' Bout', '')


    labels = details.find_all('i', class_='b-fight-details__text-item')
    for label in labels:
        desc = label.text.strip().split(':', 1)
        if desc[0].lower() == 'round':
            round_end = desc[1].lstrip()
        elif desc[0].lower() == 'time':
            time_end = desc[1].lstrip()
        elif desc[0].lower() == 'time format':
            time_format = desc[1].lstrip()
        elif desc[0].lower() == 'referee':
            referee = desc[1].lstrip()


    print(opponent_id)
    print(event_name)
    print(event_date)
    print(method_of_victory)
    print(name)
    print(alias)
    print(winner_id)
    print(corner)
    print(weight)
    print(round_end)
    print(time_end)
    print(time_format)
    print(referee)


3bd86a49-3b8e-4bb7-a6b8-54a6a4f36443
UFC 315: Muhammad vs. Della Maddalena
May 10, 2025
S-DEC  
Modestas Bukauskas
The Baltic Gladiator
02e262c6-f245-4112-bc01-b2a6954f9bd9
Red
Light Heavyweight
3
5:00
3 Rnd (5-5-5)
Dan Miragliotta


In [203]:
tables = soup.find_all('table', class_='b-fight-details__table js-fight-table')
headers = tables[0].find_all('th', class_='b-fight-details__table-col')
statistics = tables[0].find_all('td', class_='b-fight-details__table-col')

In [229]:
header_str = []
for header in headers:
    header_str.append(header.text.lstrip().rstrip().replace('\n', ''))
header_str[5] = header_str[5].replace(' %', '')
header_list = header_str[0:10]
rounds_breakdown = header_str[10:]
print(header_list)
print(rounds_breakdown)

['Fighter', 'KD', 'Sig. str.', 'Sig. str. %', 'Total str.', 'Td', 'Td %', 'Sub. att', 'Rev.', 'Ctrl', 'Fighter', 'KD', 'Sig. str.', 'Sig. str. %', 'Total str.', 'Td', 'Td %', 'Sub. att', 'Rev.', 'Ctrl', 'Fighter', 'KD', 'Sig. str.', 'Sig. str. %', 'Total str.', 'Td', 'Td %', 'Sub. att', 'Rev.', 'Ctrl']
['Round 1', 'Round 2', 'Round 3']


In [227]:
stat_list = []
for stat in statistics:
    stat_list.append(stat.text.replace('\n', '').lstrip().rstrip())
print(len(stat_list))

30


In [244]:
for rounds in rounds_breakdown:
    if corner == "Red":
        pullI = 0
    else:
        pullI = 1

    if rounds == 'Round 1':
        rd1_stats = stat_list[0:10]
        for i in range(len(header_list)):
            if header_list[i] == "KD":
                r1_knockdowns = rd1_stats[i].split(' ', 1)[pullI].lstrip().rstrip()
            elif header_list[i] == "Sig. str.":
                sig_str = stat_list[i].split('  ')[pullI*5].split(' of ')
                r1_sig_strikes_landed = sig_str[0]
                r1_sig_strikes_attempted = sig_str[1]
            #check if this is broken
            elif header_list[i] == "Total str.":
                total_str = stat_list[i].split('  ')[pullI*5].split(' of ')
                r1_total_strikes_landed = total_str[0]
                r1_total_strikes_attempted = total_str[1]
                print(r1_total_strikes_landed)
                print(r1_total_strikes_attempted)
                break


12
34


In [ ]:
#initalized before fight link
opponent_id = None
event_name = None
event_date = None
method_of_victory = None


name = None
alias = None
winner_id = None
corner = None
height = None
#get weight from bout
weight = None
reach = None
stance = None
dOB = None
rounds_scheduled = None
round_end = None
time_end = None
referee = None

#totals 
r1_knockdowns = None
r1_sig_strikes_landed = None
r1_sig_strikes_attempted = None
r1_total_strikes_landed = None
r1_total_strikes_attempted = None
r1_takedowns_landed = None
r1_takedowns_attempted = None
r1_subs_attempted = None
r1_reversals = None
r1_control_time = None

r2_knockdowns = None
r2_sig_strikes_landed = None
r2_sig_strikes_attempted = None
r2_total_strikes_landed = None
r2_total_strikes_attempted = None
r2_takedowns_landed = None
r2_takedowns_attempted = None
r2_subs_attempted = None
r2_reversals = None
r2_control_time = None

r3_knockdowns = None
r3_sig_strikes_landed = None
r3_sig_strikes_attempted = None
r3_total_strikes_landed = None
r3_total_strikes_attempted = None
r3_takedowns_landed = None
r3_takedowns_attempted = None
r3_subs_attempted = None
r3_reversals = None
r3_control_time = None

r4_knockdowns = None
r4_sig_strikes_landed = None
r4_sig_strikes_attempted = None
r4_total_strikes_landed = None
r4_total_strikes_attempted = None
r4_takedowns_landed = None
r4_takedowns_attempted = None
r4_subs_attempted = None
r4_reversals = None
r4_control_time = None

r5_knockdowns = None
r5_sig_strikes_landed = None
r5_sig_strikes_attempted = None
r5_total_strikes_landed = None
r5_total_strikes_attempted = None
r5_takedowns_landed = None
r5_takedowns_attempted = None
r5_subs_attempted = None
r5_reversals = None
r5_control_time = None


#significant strikes
r1_head_sig_strikes_landed = None
r1_head_sig_strikes_attempted = None
r1_body_sig_strikes_landed = None
r1_body_sig_strikes_attempted = None
r1_leg_sig_strikes_landed = None
r1_leg_sig_strikes_attempted = None
r1_distance_sig_strikes_landed = None
r1_distance_sig_strikes_attempted = None
r1_clinch_sig_strikes_attempted = None
r1_clinch_sig_strikes_landed = None
r1_ground_sig_strikes_landed = None
r1_ground_sig_strikes_attempted = None

r2_head_sig_strikes_landed = None
r2_head_sig_strikes_attempted = None
r2_body_sig_strikes_landed = None
r2_body_sig_strikes_attempted = None
r2_leg_sig_strikes_landed = None
r2_leg_sig_strikes_attempted = None
r2_distance_sig_strikes_landed = None
r2_distance_sig_strikes_attempted = None
r2_clinch_sig_strikes_attempted = None
r2_clinch_sig_strikes_landed = None
r2_ground_sig_strikes_landed = None
r2_ground_sig_strikes_attempted = None

r3_head_sig_strikes_landed = None
r3_head_sig_strikes_attempted = None
r3_body_sig_strikes_landed = None
r3_body_sig_strikes_attempted = None
r3_leg_sig_strikes_landed = None
r3_leg_sig_strikes_attempted = None
r3_distance_sig_strikes_landed = None
r3_distance_sig_strikes_attempted = None
r3_clinch_sig_strikes_attempted = None
r3_clinch_sig_strikes_landed = None
r3_ground_sig_strikes_landed = None
r3_ground_sig_strikes_attempted = None

r4_head_sig_strikes_landed = None
r4_head_sig_strikes_attempted = None
r4_body_sig_strikes_landed = None
r4_body_sig_strikes_attempted = None
r4_leg_sig_strikes_landed = None
r4_leg_sig_strikes_attempted = None
r4_distance_sig_strikes_landed = None
r4_distance_sig_strikes_attempted = None
r4_clinch_sig_strikes_attempted = None
r4_clinch_sig_strikes_landed = None
r4_ground_sig_strikes_landed = None
r4_ground_sig_strikes_attempted = None

r5_head_sig_strikes_landed = None
r5_head_sig_strikes_attempted = None
r5_body_sig_strikes_landed = None
r5_body_sig_strikes_attempted = None
r5_leg_sig_strikes_landed = None
r5_leg_sig_strikes_attempted = None
r5_distance_sig_strikes_landed = None
r5_distance_sig_strikes_attempted = None
r5_clinch_sig_strikes_attempted = None
r5_clinch_sig_strikes_landed = None
r5_ground_sig_strikes_landed = None
r5_ground_sig_strikes_attempted = None